In [ ]:
%matplotlib notebook
%precision 3

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pytz
import math
import tikzplotlib
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 7]
import datetime
import pandas
import scipy
import scipy.special
import numpy as np
import urllib.request
import json
from sqlalchemy import create_engine
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, LogLocator, FixedLocator, FixedFormatter, NullLocator)
from matplotlib.dates import DateFormatter
from matplotlib.animation import FuncAnimation
from datetime import timedelta

import itertools

import matplotlib.animation as animation

matplotlib.rcParams['timezone'] = 'Europe/Amsterdam'

In [ ]:
engine=create_engine("sqlite:////home/ahu/git/hello-dl/threeorseven.sqlite3")
#engine=create_engine("sqlite:////home/ahu/git/hello-dl/yo.sqlite3")
scores = pandas.read_sql_query("SELECT * from data", engine)


In [ ]:
plt.figure()
plt.hist(scores[scores.label==3].res, bins=75, lw=2, histtype='step', density=True, label='3')
plt.hist(scores[scores.label==7].res, bins=75, lw=2, histtype='step', density=True, label='7')
plt.xlabel("Score")
plt.ylabel("Density")
plt.legend()
plt.grid()
plt.savefig("linear-histo.svg")

In [ ]:
engine2=create_engine("sqlite:////home/ahu/git/hello-dl/37learn.sqlite3")
scores2 = pandas.read_sql_query("SELECT * from data", engine2)


In [ ]:
plt.figure()
plt.hist(scores2[scores2.label==3].res, bins=75, lw=2, histtype='step', density=True, label='3')
plt.hist(scores2[scores2.label==7].res, bins=75, lw=2, histtype='step', density=True, label='7')
plt.xlabel("Score")
plt.ylabel("Density")
plt.legend()
plt.grid()
plt.savefig("linear-learned-histo.svg")

In [ ]:
scores2.groupby(["label", "verdict"]).count()

In [ ]:
x = np.linspace(1, 4.5, 1000)
y = x*np.cos(x)
#dy = -np.sin(x)*x + np.cos(x)
plt.figure(figsize=(6,4))
plt.plot(x, y, lw=2,label = "$ y = x\cdot{}cos(x)$")
#plt.plot(x, dy, label = "$dy/dx$")

def dydx(x):
    return -math.sin(x)*x + math.cos(x)

lex=1.0
for c in range(0,10): 
    plt.arrow(lex, lex*math.cos(lex), -0.2 * dydx(lex), -0.2*dydx(lex)*dydx(lex), width=0.025, color='black', length_includes_head=True)
    lex = lex -0.2*dydx(lex)
    #plt.arrow(lex+0.1, lex*math.cos(lex), 0, 0.1*(-math.sin(lex)*lex + math.cos(lex)), width=0.025)
    
plt.xlabel("x")    
plt.ylabel("y")
plt.legend()
plt.grid()
plt.title("One dimensional hill climbing using 0.1 of the derivative")
plt.savefig("hill-climbing.svg")

In [ ]:
enginerelu=create_engine("sqlite:////home/ahu/git/hello-dl/tensor-relu.sqlite3")

training = pandas.read_sql_query("SELECT * from training", enginerelu)
validation = pandas.read_sql_query("SELECT * from validation", enginerelu)
confusion = pandas.read_sql_query("SELECT * from confusion", enginerelu)


In [ ]:
plt.figure(figsize=(6,4))

plt.plot(training.batchno, training.corperc, label="training")
plt.plot(validation.batchno, validation.corperc, label="validation")
plt.grid()
plt.xlabel("Batch number")
plt.ylabel("%")
plt.legend()
plt.title("Percentage of digits classified correctly")
plt.savefig("tensor-relu.svg")

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(training.batchno, training.avgloss, label="training")
plt.plot(validation.batchno, validation.avgloss, label="validation")
plt.grid()
plt.xlabel("Batch number")
plt.ylabel("loss")
plt.legend()
plt.title("Average loss")
plt.savefig("tensor-relu-loss.svg")

In [ ]:
enginerelubef=create_engine("sqlite:////home/ahu/git/hello-dl/tensor-relu-before.sqlite3")

enginerelu=create_engine("sqlite:////home/ahu/git/hello-dl/tensor-relu.sqlite3")

training = pandas.read_sql_query("SELECT * from training", enginerelu)
validation = pandas.read_sql_query("SELECT * from validation", enginerelu)
validationbef = pandas.read_sql_query("SELECT * from validation", enginerelubef)


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(training.batchno, training.corperc, label="training")
plt.plot(validation.batchno, validation.corperc, label="validation, slightly changed")
plt.plot(validationbef.batchno, validationbef.corperc, label="validation")
plt.grid()
plt.xlabel("Batch number")
plt.ylabel("%")
plt.legend()
plt.title("Percentage of digits classified correctly")
plt.savefig("slightly-moved.svg")

In [ ]:
confusion

In [ ]:
mat = []
for c in range(0,10):
    mat.append([])
    for r in range(0,10):
        mat[c].append(confusion[(confusion.r==r) & (confusion.c==c)]["count"].sum())
        
mat

In [ ]:
plt.figure()
plt.imshow(mat)

In [ ]:
df = pandas.DataFrame(mat)
print(df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys').format(precision=0).to_html())

In [ ]:
x=np.linspace(-3,3,100)
plt.figure(figsize=(6,4))
y1=0.5*x*(1 + scipy.special.erf(x/np.sqrt(2)))
plt.plot(x, y1, label="gelu")
plt.xlabel("x")
plt.ylabel("y")
plt.axhline(0, color='black', linewidth=1)
plt.axvline(0, color='black', linewidth=1)

plt.grid()
plt.legend()
plt.title("Gaussian Error \"Linear\" Unit")
plt.savefig("gelu.svg")



In [ ]:
l=pandas.read_csv("/home/ahu/git/hello-dl/l.csv", names=["idx", "val"])
l.set_index("idx", inplace=True)
plt.figure()
plt.title("Loss of GRU language model")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.plot(100*l.val.index/245448, l.val.rolling(10).mean())
plt.yscale("log")
plt.grid()


In [ ]:
enginerelu=create_engine("sqlite:////home/ahu/git/hello-dl/convo-vals.sqlite3")

training = pandas.read_sql_query("SELECT * from training", enginerelu)
validation = pandas.read_sql_query("SELECT * from validation", enginerelu)
#confusion = pandas.read_sql_query("SELECT * from confusion", enginerelu)
training = training[training.startID==1676748833]
validation = validation[validation.startID==1676748833]

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(training.elapsed/3600, training.corperc.rolling(30).mean(), label="training")
plt.plot(validation.elapsed/3600, validation.corperc.rolling(10).mean(), label="validation")
plt.grid()
plt.xlabel("Batch number")
plt.ylabel("%")
plt.legend()
plt.title("Percentage of digits classified correctly")
plt.savefig("tensor-convo-sgd.svg")

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(training.batchno, training.avgloss, label="training")
plt.plot(validation.batchno, validation.avgloss, label="validation")
plt.grid()
plt.xlabel("Batch number")
plt.ylabel("loss")
plt.legend()
plt.title("Average loss")
plt.savefig("tensor-relu-loss.svg")

In [ ]:
p = pandas.read_csv("/home/ahu/git/hello-dl/plot.csv", sep="\t")

In [ ]:
plt.figure()
plt.plot(p.num, p.val.rolling(10).mean())


In [ ]:
# set parameters
frames = range(0, 20)

# set how the graph will change each frame

# init the figure
fig, ax = plt.subplots(figsize=(6,4))

x = np.linspace(1, 4.5, 1000)
y = x*np.cos(x) 
def dydx(x):
    return -math.sin(x)*x + math.cos(x)


def update(i):
    # clear the axis each frame
    ax.clear()
    ax.set_xlim(0.7,4.8)
    ax.set_ylim(-4.5,1)
    ax.grid()
    # replot things
    ax.plot(x, y, lw=2,label = "$ y = x\cdot{}cos(x)$")

    lex=1.0
    for c in range(0,i): 
        ax.arrow(lex, lex*math.cos(lex) , -0.2 * dydx(lex), -0.2*dydx(lex)*dydx(lex), width=0.025, color='black',
                 length_includes_head=True)
        lex = lex -0.2*dydx(lex)

    ax.legend()
    ax.set_title("Gradient descent without momentum")          
    # reformat things
    ax.set_xlabel('x')
    ax.set_ylabel('y')

ani = animation.FuncAnimation(fig, update, frames=frames, interval=500)
ani.save('sgd.gif', writer='imagemagick', fps=3)

In [ ]:
# set parameters
frames = range(0, 20)

# set how the graph will change each frame

# init the figure
fig, ax = plt.subplots(figsize=(6,4))
alpha=0.225
x = np.linspace(0.5, 4.5, 1000)
y = x*np.cos(x) - alpha*np.sin(6*x)

def dydx(x):
    return -math.sin(x)*x + math.cos(x) - 6*alpha*np.cos(6*x)


def update(i):
    # clear the axis each frame
    ax.clear()
    ax.set_xlim(0.5,4.8)
    ax.set_ylim(-4.5,1)
    ax.grid()
    # replot things
    ax.plot(x, y, lw=2,label = "$ y = x\cdot{}cos(x) -0.2\cdot{}sin(6x)$")

    lex=0.85
    v=0
    for c in range(0,i): 
        #ax.arrow(lex, lex*math.cos(lex) - alpha*math.sin(6*lex), -0.2 * dydx(lex), -0.2*dydx(lex)*dydx(lex), width=0.025, color='black',
        #         length_includes_head=True)
        
        v = 0.1 * v + 0.9*dydx(lex)
        
        ax.arrow(lex, lex*math.cos(lex) - alpha*math.sin(6*lex), -0.2 * v, -0.2*v*v, width=0.025, color='black',
                 length_includes_head=True)

        #lex = lex -0.2*dydx(lex)
        lex = lex -0.2*v

    ax.legend()
    ax.set_title("Gradient descent with momentum")
    # reformat things
    ax.set_xlabel('x')
    ax.set_ylabel('y')

ani = animation.FuncAnimation(fig, update, frames=frames, interval=500)
ani.save('sgd-complex-momentum.gif', writer='imagemagick', fps=3)

In [ ]:
x = np.linspace(1, 4.5, 1000)
y = x*np.cos(x)
#dy = -np.sin(x)*x + np.cos(x)
plt.figure(figsize=(6,4))
plt.plot(x, y, lw=2,label = "$ y = x\cdot{}cos(x)$")
#plt.plot(x, dy, label = "$dy/dx$")

def dydx(x):
    return -math.sin(x)*x + math.cos(x)

lex=1.0
for c in range(0,10): 
    plt.arrow(lex, lex*math.cos(lex), -0.2 * dydx(lex), -0.2*dydx(lex)*dydx(lex), width=0.025, color='black', length_includes_head=True)
    lex = lex -0.2*dydx(lex)
    #plt.arrow(lex+0.1, lex*math.cos(lex), 0, 0.1*(-math.sin(lex)*lex + math.cos(lex)), width=0.025)
    
plt.xlabel("x")    
plt.ylabel("y")
plt.legend()
plt.grid()
plt.title("One dimensional hill climbing using 0.1 of the derivative")
plt.savefig("hill-climbing.svg")